# Power user data

Setup

In [1]:
%load_ext autoreload
%autoreload 2

from hereutil import here, add_to_sys_path
add_to_sys_path(here())

from src.common_basis import *
import pandas as pd


In [5]:
get_s3fs().ls('/dhh24/disc/parquet')


['dhh24/disc/parquet/aita_comments_sample_1_a.parquet',
 'dhh24/disc/parquet/aita_submissions_a.parquet',
 'dhh24/disc/parquet/aita_submissions_sample_1_a.parquet',
 'dhh24/disc/parquet/best_of_reddit_comments_sample_1_a.parquet',
 'dhh24/disc/parquet/best_of_reddit_submissions_a.parquet',
 'dhh24/disc/parquet/best_of_reddit_submissions_sample_1_a.parquet',
 'dhh24/disc/parquet/cmw_comments_a.parquet',
 'dhh24/disc/parquet/cmw_comments_sample_1_a.parquet',
 'dhh24/disc/parquet/cmw_delta_comments_a.parquet',
 'dhh24/disc/parquet/cmw_submissions_a.parquet',
 'dhh24/disc/parquet/cmw_submissions_sample_1_a.parquet',
 'dhh24/disc/parquet/eli5_comments_a.parquet',
 'dhh24/disc/parquet/eli5_comments_sample_1_a.parquet',
 'dhh24/disc/parquet/eli5_submissions_a.parquet',
 'dhh24/disc/parquet/eli5_submissions_sample_1_a.parquet',
 'dhh24/disc/parquet/power_user_aita_comments.parquet',
 'dhh24/disc/parquet/power_user_cmv_all_comments.parquet',
 'dhh24/disc/parquet/power_user_cmv_delta_comments.pa

Get delta awarding comments:

In [2]:
delta_awarding_comments = pd.read_parquet( 'dhh24/disc/parquet/cmw_delta_comments_a.parquet', filesystem=get_s3fs(),engine="pyarrow")


Get all cmv comments:

In [3]:
cmv = pd.read_parquet('dhh24/disc/parquet/cmw_comments_a.parquet', filesystem=get_s3fs(),engine="pyarrow")


Find delta-awarded comments and find the users who have earned deltas. Find power users:

In [4]:
delta_awarded_comments = cmv[cmv['id'].isin(delta_awarding_comments['parent_comment_id'])]
# using author because ther eare so many missing author_ids
# if 'author' (username) belongs to more than one user_id this will make problems
power_users = delta_awarded_comments.groupby('author')['author'].value_counts() 
power_users = power_users[power_users > 150]
power_users = power_users[power_users.index != "[deleted]"]
power_users # no deleted users


author
10ebbor10               184
47ca05e6209a317a8fb3    159
A_Soporific             162
Amablue                 184
Ansuz07                 630
                       ... 
scottevil110            177
tbdabbholm              186
themcos                 349
thethoughtexperiment    271
yyzjertl                483
Name: count, Length: 67, dtype: int64

Find all delta-awarded comments made by power users:

In [5]:
power_user_data = delta_awarded_comments.loc[delta_awarded_comments['author'].isin(power_users.index)]
power_user_data

,subreddit_id,subreddit,id,permalink,link_id,parent_comment_id,created_utc,author_id,author,body,score,controversiality
2,4855832,changemyview,32388170293,https://www.reddit.com/r/changemyview/comments...,759936720,3.238813e+10,2019-07-31 21:03:13,6.164020e+06,ralph-j,&gt; I feel like that shouldn't be a thing peo...,16,False
67,4855832,changemyview,32388363965,https://www.reddit.com/r/changemyview/comments...,759968996,NaN,2019-07-31 21:36:41,5.410853e+06,AnythingApplied,Even if we stopped producing carbon dioxide ri...,13,False
251,4855832,changemyview,32388834566,https://www.reddit.com/r/changemyview/comments...,760019455,NaN,2019-07-31 23:01:51,9.836526e+06,cdb03b,It was war. Total war at that which is a form ...,6,False
498,4855832,changemyview,32389404872,https://www.reddit.com/r/changemyview/comments...,760044185,NaN,2019-08-01 00:58:17,5.410853e+06,AnythingApplied,Most people aren't generally open to changing ...,49,False
889,4855832,changemyview,32390792865,https://www.reddit.com/r/changemyview/comments...,760170300,NaN,2019-08-01 06:20:01,1.610848e+07,MercurianAspirations,I think it's disingenuous to say that politici...,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...
13373087,4855832,changemyview,36125883385,https://www.reddit.com/r/changemyview/comments...,1283865812,3.612162e+10,2021-01-31 04:22:38,4.973701e+07,thethoughtexperiment,&gt; Ok. I think I can understand that regardi...,3,False
13373915,4855832,changemyview,36127823765,https://www.reddit.com/r/changemyview/comments...,1285585171,NaN,2021-01-31 13:42:04,9.420607e+06,I_am_the_night,"So I think in general you are right, a lot of ...",12,False
13374076,4855832,changemyview,36128115638,https://www.reddit.com/r/changemyview/comments...,1285587395,NaN,2021-01-31 14:33:34,5.410853e+06,AnythingApplied,This is the exact reason why you shouldn't get...,22,False
13374704,4855832,changemyview,36129124320,https://www.reddit.com/r/changemyview/comments...,1285732443,3.612911e+10,2021-01-31 17:18:51,1.744073e+11,miguelguajiro,But I’ve shown you the evidence and explained ...,3,False


Save the data to a parquet:

In [ ]:
power_user_data.to_parquet('dhh24/disc/parquet/power_user_cmv_delta_comments.parquet', filesystem=get_s3fs(),engine="pyarrow")

Get and save all comments made by power users:

In [2]:
power_users = pd.read_parquet('dhh24/disc/parquet/power_user_cmv_delta_comments.parquet', filesystem=get_s3fs(),engine="pyarrow", columns=['author'])
power_users = power_users['author'].unique()
fil = [('author', "in", power_users)]

In [4]:
aita = pd.read_parquet('dhh24/disc/parquet/t_aita_comments_a.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
aita.to_parquet('dhh24/disc/parquet/power_user_aita_comments.parquet', filesystem=get_s3fs(),engine="pyarrow")

(67,)

In [7]:
cmv = pd.read_parquet('dhh24/disc/parquet/cmw_comments_a.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
cmv.to_parquet('dhh24/disc/parquet/power_user_cmv_all_comments.parquet', filesystem=get_s3fs(),engine="pyarrow")


In [3]:
eli = pd.read_parquet('dhh24/disc/parquet/eli5_comments_a.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
eli.to_parquet('dhh24/disc/parquet/power_user_eli5_comments.parquet', filesystem=get_s3fs(),engine="pyarrow")

In [4]:
uo = pd.read_parquet('dhh24/disc/parquet/unpopularopinion_comments_a.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
uo.to_parquet('dhh24/disc/parquet/power_user_unpopularopinion_comments.parquet', filesystem=get_s3fs(),engine="pyarrow")

To tsv:

In [ ]:
uo = pd.read_parquet('dhh24/disc/parquet/power_user_unpopularopinion_comments.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
uo.to_csv('../../data/work/samples/power_user_unpopularopinion_comments.tsv', sep='\t')


In [ ]:
cmv = pd.read_parquet('dhh24/disc/parquet/power_user_cmv_all_comments.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
cmv.to_csv('../../data/work/samples/power_user_cmv_comments.tsv', sep='\t')

In [ ]:
aita = pd.read_parquet('dhh24/disc/parquet/power_user_aita_comments.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
aita.to_csv('../../data/work/samples/power_user_aita_comments.tsv', sep='\t')

In [ ]:
eli5 = pd.read_parquet('dhh24/disc/parquet/power_user_eli5_comments.parquet', filesystem=get_s3fs(),engine="pyarrow", filters=fil)
eli5.to_csv('../../data/work/samples/power_user_eli_comments.tsv', sep='\t')